<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Credit Risk Assessment using Teradataml OpenSource Functions  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Credit risk assessment is the process of evaluating the likelihood that a borrower will default on a financial obligation, such as repaying a loan or meeting other credit obligations. It involves analyzing various factors related to the borrower's financial health, credit history, and overall ability to repay debt. The primary goal of credit risk assessment is to enable lenders, such as banks, credit unions, or lending institutions, to make informed decisions about whether to extend credit to an individual or entity and under what terms.</p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Value</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'> One of the primary objectives of credit risk assessment is to minimize the risk of default on loans or credit lines. </li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Effective credit risk assessment enables financial institutions to make more informed lending decisions. By evaluating factors such as the borrower's credit history, income, debt levels, and other relevant data, institutions can tailor loan terms and interest rates to reflect the level of risk associated with each borrower.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>By implementing robust credit risk assessment processes, institutions can ensure compliance with regulatory mandates such as Basel III, which requires banks to maintain adequate capital reserves based on the risk profiles of their lending portfolios. </li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>By accurately assessing credit risk, institutions can offer competitive loan terms to creditworthy borrowers, thereby facilitating access to credit for individuals and businesses with demonstrated repayment capacity.</li>
    </p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Vantage's ClearScape Analytics provides a comprehensive library of in-database (inDb) functions tailored for various stages of the data science lifecycle, including data exploration, preparation, feature engineering, model training, and evaluation. These functions seamlessly execute within the Vantage SQL engine, ensuring high performance and scalability while eliminating the need for data movement.<br>By leveraging the Teradata Python Package, specifically the teradataml OpenSourceML component, users gain the ability to utilize popular open-source machine learning packages like scikit-learn directly within the database environment. This means that data doesn't need to be transferred to the client for analysis, streamlining the workflow and enhancing efficiency.<br>The OpenSourceML package simplifies the integration of open-source machine learning functionalities into Vantage, offering a consistent interface for executing these algorithms. Users can leverage familiar syntax and arguments, making it easier to transition from traditional open-source environments to the Vantage platform.<br>Running these machine learning functions within Vantage enables organizations to scale their models effectively. Horizontal scaling allows for the training of segmented models based on various criteria such as region or user type. Meanwhile, vertical scaling enables the consolidation of data from millions or even billions of interactions, empowering organizations to leverage vast datasets for model training and analysis.<br>Furthermore, the models developed using these functionalities can be seamlessly deployed for operational use, enabling real-time understanding and prediction of actions. This operational deployment capability ensures that organizations can derive actionable insights and make informed decisions in a timely manner, driving value across various business domains.</p> 

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us start by checking the version of the teradataml installed. The Openml functions used in this notebook will require Version 20.0.0.0.</p>

In [ ]:
pip show teradataml

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>If the VM has lower version, please uncomment the below code and execute the cell.  After the cell executes, please restart the kernel. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
#!pip install --upgrade teradataml

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Import the required libraries, set environment variables and connect to Vantage.</p>

In [ ]:
#import libraries
import getpass
import warnings


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

from teradataml import *


display.max_rows = 5

# Suppress warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys. </p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys.</p>

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Credit_Risk_Assessment_Python.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.
</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_CreditRisk_cloud');"
# takes about 1 minute, estimated space: 0 MB
# %run -i ../run_procedure.py "call get_data('DEMO_CreditRisk_local');"
# takes about 2 minutes, estimated space: 23 MB

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Analyze the data set</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Let us start by analyzing the credit data of customers.

In [ ]:
tdf = DataFrame(in_schema('DEMO_CreditRisk', 'Credit_Customers'))
tdf

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Checking the demographics of data</p>

In [ ]:
tdf.shape

In [ ]:
tdf.info()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can see from above that we have 1000 records with 23 columns.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C;color:#00233C'><b>Summary of columns</b><br>
<p style = 'font-size:16px;font-family:Arial;color:#00233C;'>The <b>ColumnSummary</b> function can be used to take a quick look at the columns, their datatypes, and summary of NULLs/non-NULLs for a given table.   </p>

In [ ]:
colsum = ColumnSummary(data=tdf,
                        target_columns=[':']
                       )
colsum.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C;color:#00233C'><b>Checking the null percenatge of columns</b></p>

In [ ]:
cs = colsum.result.filter(items = ['ColumnName', 'Datatype', 'NullPercentage'])
cs[cs['NullPercentage'] > 0.0]

<p style = 'font-size:16px;font-family:Arial;color:#00233C;'>From above we can see that there is no nulls in the data.<br>Let us know create lists of features which are categorical and numerical, we will use them in functions later.</p>

In [ ]:
# Create a list of column names with data type 'str'
str_cols = [col.split()[0] for col in str(tdf.dtypes).split('\n') if col.split()[1] == 'str']
str_cols

In [ ]:
# Create a list of column names with data type not 'str'
num_cols = [col.split()[0] for col in str(tdf.dtypes).split('\n') if col.split()[1] != 'str']
num_cols

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Data Preprocessing</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Before the data can be used for model creation, we will need to do some data cleansing and transformation on it. We can do this InDb with Teradata Vantage's inbuilt functions.<br>The <b>CategoricalSummary</b> function displays the distinct values and their counts for each specified input DataFrame column.</p>

In [ ]:
catsum = CategoricalSummary(data=tdf,
                             target_columns=str_cols
                            )
 
catsum.result

 <p style = 'font-size:16px;font-family:Arial;color:#00233C'>Checking the number of distinct values in each of categorical columns.</p>

In [ ]:
c1=catsum.result
count_column = c1.DistinctValue.count(distinct=True)

d2=c1.groupby("ColumnName").assign(count_=count_column)
d2.sort('count_').head(20)

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Ordinal Encoding</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now let us convert the string categorical values to numerical values. We can do this by using <b>Clearscape Analytics OrdinalEncoding Fit and Transform </b>functions. The Fit function creates a table which gives numerical value to each of the category of the column and the Transform function will replace the value of category string to its assigned numerical value.</p>

In [ ]:
# Perform OrdinalEncoding
ordinal_obj = OrdinalEncodingFit(target_column=str_cols,
                                 data=tdf)

In [ ]:
ordinal_obj.result

In [ ]:
# Transforming the encoded data.
tdf2 = ordinal_obj.transform(data=tdf,
                     accumulate=num_cols).result

tdf2

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Feature Engineering</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will create 2 new features to add to our dataset for model training.<br>We will add below new columns
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li>debt_to_income_ratio</li><li>credit_utilization</li></ul></p>


In [ ]:
tdf3 = tdf2.assign(debt_to_income_ratio = tdf2.credit_amount/tdf2.duration * tdf2.installment_commitment,
                  credit_utilization = tdf2.credit_amount/ tdf2.existing_credits)
tdf3

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Bin Encoding</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now let us convert the age in age groups. We can do this by using <b>Clearscape Analytics BinEncoding Fit and Transform </b>functions. Let us first create the fit table which divides age in different groups.</p>

In [ ]:
# Define the data as a dictionary
data = {
    'ColumnName': ['age', 'age', 'age', 'age', 'age','age','age'],
    'MinValue': [0,21,31,41,51,61,71],
    'MaxValue': [20,30,40,50,60,70,150],
    'Label': ['0-20', '21-30', '31-40', '41-50', '51-60','61-70','70+'],
    'Label_Added' :['1','2','3','4','5','6','7']
}


# Create the DataFrame
bin_fit_df = pd.DataFrame(data)
bin_fit_df

#copy the dataframe to Vantage
copy_to_sql(bin_fit_df, table_name='bin_fit', schema_name='demo_user', if_exists='replace')
bin_fit = DataFrame('bin_fit')

In [ ]:
#create the Bin Fit table
bin_fit = BincodeFit(data=tdf3,
                            fit_data=bin_fit,
                            fit_data_order_column = ['MinValue', 'MaxValue'],
                            target_columns='age',
                            minvalue_column='MinValue',
                            maxvalue_column='MaxValue',
                            label_column='Label_Added',
                            method_type='Variable-Width',
                            label_prefix='label_prefix'
                           )
 
# Print the result.
bin_fit.output

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From above you can see the fit table that was created by BinEncodingFit function. Now let us apply this to our data.</p>

In [ ]:
acc_col = tdf3.columns
acc_col.remove('age')

#Transform the data
obj = BincodeTransform(data=tdf3,
                           object=bin_fit.output,
                           object_order_column="TD_MinValue_BINFIT",
                           accumulate=acc_col
                          )
 
#display result
obj.result

In [ ]:
td4=obj.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>ScaleFit and ScaleTransform function scales specified input table columns i.e perform the specific scale methods like standard deviation, mean etc to the input columns </p>

In [ ]:
#create the Scale fit table
sf_fit = ScaleFit(
    data = td4,
    scale_method = 'RANGE',
    target_columns = ['duration', 'credit_amount', 'installment_commitment', 'residence_since',
            'debt_to_income_ratio' ,'credit_utilization', 'existing_credits' ])

In [ ]:
str_cols.append("age")
str_cols.append("id")

#Transform the data
res = sf_fit.transform(
    data = td4,
    accumulate = str_cols).result

#display result
res

In [ ]:
# Create the cleansed data table in the database
copy_to_sql(res, table_name = 'clean_data', if_exists = 'replace')


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Model Creation</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us take a look at our data that we have cleansed and processed above.</p>

In [ ]:
tdf = DataFrame('clean_data')
tdf

In [ ]:
tdf.shape

In [ ]:
tdf = tdf.assign(age = tdf.age.cast(type_ = INTEGER))

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Train and Test Datasets</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now let us divide our data in training and test datasets for model creation. We can do this by using Clearscape Analytics TrainTestSplit function.

In [ ]:
TrainTestSplit_out = TrainTestSplit(
                                    data = tdf,
                                    id_column = "id",
                                    train_size = 0.80,
                                    test_size = 0.20,
                                    seed = 25
                                   )

In [ ]:
df_train = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 1].drop(['TD_IsTrainRow'], axis = 1)
df_test = TrainTestSplit_out.result[TrainTestSplit_out.result['TD_IsTrainRow'] == 0].drop(['TD_IsTrainRow'], axis = 1)

In [ ]:
df_train

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Divide the datasets in features and output label to be used in model training functions.

In [ ]:
X_train = df_train.drop(['output_class','id'], axis = 1)
y_train = df_train.select(["output_class"])
X_test = df_test.drop(['output_class','id'], axis = 1)
y_test = df_test.select(["output_class"])

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 DecisionTree</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Decision trees are a popular and intuitive machine learning algorithm used for both classification and regression tasks. Decision trees are a machine learning algorithm that organizes data into a tree-like structure, where each internal node represents a decision based on a feature, and each leaf node represents the outcome. By recursively splitting the data based on the most informative features, decision trees can efficiently partition the dataset into homogeneous subsets. They offer interpretability, as the tree structure can be easily understood, and implicit feature selection, revealing which features are most important for prediction. <br> With the addition of teradataml OpenSourceML functions we can seamlessly use the DecisionTreeClassifier available in scikit-learn python module as an inDb function.</p>


In [ ]:
# Instantiate scikit-learn DecisionTreeClassifier object through teradataml OpenSourceML's 'td_sklearn'.
from teradataml import td_sklearn as osml
DT_classifier = osml.DecisionTreeClassifier(random_state=42)
DT_classifier.fit(X_train, y_train)

In [ ]:
#accuracy of the model
accuracy_DT = DT_classifier.score(X_test, y_test)
accuracy_DT

In [ ]:
#model predictions
predict_DT =DT_classifier.predict(X_test,y_test)
predict_DT

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1.1 Confusion Matrix for DecisionTree</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A confusion matrix is a matrix that summarizes the performance of a machine learning model on a set of test data. It is a means of displaying the number of accurate and inaccurate instances based on the model’s predictions. It is often used to measure the performance of classification models, which aim to predict a categorical label for each input instance.<br>Let us create the confusion matrix for the DecisionTreeClassifier we have used above.</p>

In [ ]:
cm_DT = osml.confusion_matrix(y_true=predict_DT.select(["output_class"]),y_pred=predict_DT.select(["decisiontreeclassifier_predict_1"]))

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can cross check the values our encoder has given to output_class 'Good' and 'Bad'</p>

In [ ]:
ordinal_obj.result.select(["TD_ColumnName_ORDFIT","TD_Category_ORDFIT","TD_Value_ORDFIT"])[ordinal_obj.result['TD_ColumnName_ORDFIT'] == 'output_class']

In [ ]:
# Define class labels
class_names = ['Bad', 'Good']

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_DT, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 K-Nearest Neighbors</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'> Let us check one more classifier for our data.<br>KNN is a non-parametric, supervised learning classifier. It uses the proximity of data points to make predictions or classifications. Given an input data point, KNN identifies the k nearest neighbors (other data points) based on a chosen distance metric (usually Euclidean distance). The class label of the input point is determined by a majority vote among its k neighbors.<br> We will use scikit-learn's KNeighborsClassifier for this</p>


In [ ]:
#define the classifier
KN_classifier = osml.KNeighborsClassifier()
KN_classifier.fit(X_train, y_train)

In [ ]:
#accuracy of the model
accuracy_KN = KN_classifier.score(X_test, y_test)
accuracy_KN

In [ ]:
#model predictions
predict_KN =KN_classifier.predict(X_test,y_test)
predict_KN

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2.1 Confusion Matrix for KNN</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us create the confusion matrix for the KNeighborsClassifier we have used above.</p>

In [ ]:
cm_knn = osml.confusion_matrix(y_true=predict_KN.select(["output_class"]),y_pred=predict_KN.select(["kneighborsclassifier_predict_1"]))

In [ ]:
# Define class labels
class_names = ['Bad', 'Good']

# Display confusion matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm_knn, display_labels=class_names)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for KNN classifier')
plt.show()

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this notebook we have seen the seamless integration of Teradata Vantage and ClearScape Analytics inDb functions with OpensourceML functions, empowering users with scalable data processing capabilities. By leveraging these functionalities within the database environment, users gain the flexibility to analyze vast amounts of data without the need to transfer it to client tools. This integration not only streamlines the workflow but also provides users with the choice of utilizing open-source machine learning functions directly within the database, further enhancing efficiency and expanding the range of analytical tools available.</p>

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>6. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Work tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We need to clean up our work tables to prevent errors next time.</p>

In [ ]:
db_drop_table(table_name='clean_data') 

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Database and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_CreditRisk');" 
#Takes 10 seconds

In [ ]:
remove_context()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If you have updated the teradataml package, reinstall the package by uncommenting and running the below code cell.</p>

In [ ]:
#!pip install teradataml==17.20.0.6 --force-reinstall

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Filters: </b></p>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Industry:</b> Financial</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Functionality:</b> Open-Source Machine Learning Functions</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use Case:</b> Credit Risk Assessment</li></p>
    <p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Related Resources:</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://www.teradata.com/customers/swedbank-cloud-modernization'>How Swedbank Builds Trust Using Cloud Analytics and Data</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://www.teradata.com/Blogs/Fraud-Busting-AI'>Fraud-Busting-AI</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://www.teradata.com/Industries/Financial-Services'>Financial Services</a></li>


<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            © 2024 Teradata. All rights reserved.
        </div>
    </div>
</footer>